In [1]:
# Copyright (c) OpenMMLab. All rights reserved.
import mimetypes
import os
import time
from argparse import ArgumentParser

import cv2
import json_tricks as json
import mmcv
import mmengine
import numpy as np

from mmpose.apis import inference_topdown
from mmpose.apis import init_model as init_pose_estimator
from mmpose.evaluation.functional import nms
from mmpose.registry import VISUALIZERS
from mmpose.structures import merge_data_samples, split_instances
from mmpose.utils import adapt_mmdet_pipeline

try:
    from mmdet.apis import inference_detector, init_detector
    has_mmdet = True
except (ImportError, ModuleNotFoundError):
    has_mmdet = False


def process_one_image(args,
                      img,
                      detector,
                      pose_estimator,
                      visualizer=None,
                      show_interval=0):
    """Visualize predicted keypoints (and heatmaps) of one image."""
    print(args)
    # predict bbox
    det_result = inference_detector(detector, img)
    pred_instance = det_result.pred_instances.cpu().numpy()
    bboxes = np.concatenate(
        (pred_instance.bboxes, pred_instance.scores[:, None]), axis=1)
    bboxes = bboxes[np.logical_and(pred_instance.labels == args.det_cat_id,
                                   pred_instance.scores > args.bbox_thr)]
    bboxes = bboxes[nms(bboxes, args.nms_thr), :4]

    # predict keypoints
    pose_results = inference_topdown(pose_estimator, img, bboxes)
    data_samples = merge_data_samples(pose_results)

    # show the results
    if isinstance(img, str):
        img = mmcv.imread(img, channel_order='rgb')
    elif isinstance(img, np.ndarray):
        img = mmcv.bgr2rgb(img)

    if visualizer is not None:
        visualizer.add_datasample(
            'result',
            img,
            data_sample=data_samples,
            draw_gt=False,
            draw_heatmap=args.draw_heatmap,
            draw_bbox=args.draw_bbox,
            show_kpt_idx=args.show_kpt_idx,
            skeleton_style=args.skeleton_style,
            show=args.show,
            wait_time=show_interval,
            kpt_thr=args.kpt_thr)

    # if there is no instance detected, return None
    return data_samples.get('pred_instances', None)


def main():
    """Visualize the demo images.

    Using mmdet to detect the human.
    """
    parser = ArgumentParser()
    parser.add_argument('det_config', help='Config file for detection')
    parser.add_argument('det_checkpoint', help='Checkpoint file for detection')
    parser.add_argument('pose_config', help='Config file for pose')
    parser.add_argument('pose_checkpoint', help='Checkpoint file for pose')
    parser.add_argument(
        '--input', type=str, default='', help='Image/Video file')
    parser.add_argument(
        '--show',
        action='store_true',
        default=False,
        help='whether to show img')
    parser.add_argument(
        '--output-root',
        type=str,
        default='',
        help='root of the output img file. '
        'Default not saving the visualization images.')
    parser.add_argument(
        '--save-predictions',
        action='store_true',
        default=False,
        help='whether to save predicted results')
    parser.add_argument(
        '--device', default='cuda:0', help='Device used for inference')
    parser.add_argument(
        '--det-cat-id',
        type=int,
        default=0,
        help='Category id for bounding box detection model')
    parser.add_argument(
        '--bbox-thr',
        type=float,
        default=0.3,
        help='Bounding box score threshold')
    parser.add_argument(
        '--nms-thr',
        type=float,
        default=0.3,
        help='IoU threshold for bounding box NMS')
    parser.add_argument(
        '--kpt-thr',
        type=float,
        default=0.3,
        help='Visualizing keypoint thresholds')
    parser.add_argument(
        '--draw-heatmap',
        action='store_true',
        default=False,
        help='Draw heatmap predicted by the model')
    parser.add_argument(
        '--show-kpt-idx',
        action='store_true',
        default=False,
        help='Whether to show the index of keypoints')
    parser.add_argument(
        '--skeleton-style',
        default='mmpose',
        type=str,
        choices=['mmpose', 'openpose'],
        help='Skeleton style selection')
    parser.add_argument(
        '--radius',
        type=int,
        default=3,
        help='Keypoint radius for visualization')
    parser.add_argument(
        '--thickness',
        type=int,
        default=1,
        help='Link thickness for visualization')
    parser.add_argument(
        '--show-interval', type=int, default=0, help='Sleep seconds per frame')
    parser.add_argument(
        '--alpha', type=float, default=0.8, help='The transparency of bboxes')
    parser.add_argument(
        '--draw-bbox', action='store_true', help='Draw bboxes of instances')

    assert has_mmdet, 'Please install mmdet to run the demo.'

    args = parser.parse_args()

    assert args.show or (args.output_root != '')
    assert args.input != ''
    assert args.det_config is not None
    assert args.det_checkpoint is not None

    output_file = None
    if args.output_root:
        mmengine.mkdir_or_exist(args.output_root)
        output_file = os.path.join(args.output_root,
                                   os.path.basename(args.input))
        if args.input == 'webcam':
            output_file += '.mp4'

    if args.save_predictions:
        assert args.output_root != ''
        args.pred_save_path = f'{args.output_root}/results_' \
            f'{os.path.splitext(os.path.basename(args.input))[0]}.json'

    # build detector
    detector = init_detector(
        args.det_config, args.det_checkpoint, device=args.device)
    detector.cfg = adapt_mmdet_pipeline(detector.cfg)

    # build pose estimator
    pose_estimator = init_pose_estimator(
        args.pose_config,
        args.pose_checkpoint,
        device=args.device,
        cfg_options=dict(
            model=dict(test_cfg=dict(output_heatmaps=args.draw_heatmap))))

    # build visualizer
    pose_estimator.cfg.visualizer.radius = args.radius
    pose_estimator.cfg.visualizer.alpha = args.alpha
    pose_estimator.cfg.visualizer.line_width = args.thickness
    visualizer = VISUALIZERS.build(pose_estimator.cfg.visualizer)
    # the dataset_meta is loaded from the checkpoint and
    # then pass to the model in init_pose_estimator
    visualizer.set_dataset_meta(
        pose_estimator.dataset_meta, skeleton_style=args.skeleton_style)

    if args.input == 'webcam':
        input_type = 'webcam'
    else:
        input_type = mimetypes.guess_type(args.input)[0].split('/')[0]

    if input_type == 'image':

        # inference
        pred_instances = process_one_image(args, args.input, detector,
                                           pose_estimator, visualizer)
        print(pred_instances[0])
        if args.save_predictions:
            pred_instances_list = split_instances(pred_instances)

        if output_file:
            img_vis = visualizer.get_image()
            mmcv.imwrite(mmcv.rgb2bgr(img_vis), output_file)

    elif input_type in ['webcam', 'video']:

        if args.input == 'webcam':
            cap = cv2.VideoCapture(0)
        else:
            cap = cv2.VideoCapture(args.input)

        video_writer = None
        pred_instances_list = []
        frame_idx = 0

        while cap.isOpened():
            success, frame = cap.read()
            frame_idx += 1

            if not success:
                break

            # topdown pose estimation
            pred_instances = process_one_image(args, frame, detector,
                                               pose_estimator, visualizer,
                                               0.001)

            if args.save_predictions:
                # save prediction results
                pred_instances_list.append(
                    dict(
                        frame_id=frame_idx,
                        instances=split_instances(pred_instances)))

            # output videos
            if output_file:
                frame_vis = visualizer.get_image()

                if video_writer is None:
                    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
                    # the size of the image with visualization may vary
                    # depending on the presence of heatmaps
                    video_writer = cv2.VideoWriter(
                        output_file,
                        fourcc,
                        25,  # saved fps
                        (frame_vis.shape[1], frame_vis.shape[0]))

                video_writer.write(mmcv.rgb2bgr(frame_vis))

            # press ESC to exit
            if cv2.waitKey(5) & 0xFF == 27:
                break

            time.sleep(args.show_interval)

        if video_writer:
            video_writer.release()

        cap.release()

    else:
        args.save_predictions = False
        raise ValueError(
            f'file {os.path.basename(args.input)} has invalid format.')

    if args.save_predictions:
        with open(args.pred_save_path, 'w') as f:
            json.dump(
                dict(
                    meta_info=pose_estimator.dataset_meta,
                    instance_info=pred_instances_list),
                f,
                indent='\t')
        print(f'predictions have been saved at {args.pred_save_path}')


if __name__ == '__main__':
    main()


usage: ipykernel_launcher.py [-h] [--input INPUT] [--show] [--output-root OUTPUT_ROOT] [--save-predictions]
                             [--device DEVICE] [--det-cat-id DET_CAT_ID] [--bbox-thr BBOX_THR] [--nms-thr NMS_THR]
                             [--kpt-thr KPT_THR] [--draw-heatmap] [--show-kpt-idx]
                             [--skeleton-style {mmpose,openpose}] [--radius RADIUS] [--thickness THICKNESS]
                             [--show-interval SHOW_INTERVAL] [--alpha ALPHA] [--draw-bbox]
                             det_config det_checkpoint pose_config pose_checkpoint
ipykernel_launcher.py: error: the following arguments are required: det_checkpoint, pose_config, pose_checkpoint


SystemExit: 2

C:\Users\user\anaconda3\envs\openmmlab\lib\site-packages\IPython\core\interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [6]:
import mimetypes
import os
import time
from argparse import ArgumentParser

import cv2
import json_tricks as json
import mmcv
import mmengine
import numpy as np

from mmpose.apis import inference_topdown
from mmpose.apis import init_model as init_pose_estimator
from mmpose.evaluation.functional import nms
from mmpose.registry import VISUALIZERS
from mmpose.structures import merge_data_samples, split_instances
from mmpose.utils import adapt_mmdet_pipeline
from mmdet.apis import inference_detector, init_detector

In [16]:
def process_one_image(
                      img,
                      detector,
                      pose_estimator,
                      visualizer=None,
                      show_interval=0):
    """Visualize predicted keypoints (and heatmaps) of one image."""

    # predict bbox
    det_result = inference_detector(detector, img)
    pred_instance = det_result.pred_instances.cpu().numpy()
    bboxes = np.concatenate(
        (pred_instance.bboxes, pred_instance.scores[:, None]), axis=1)
    bboxes = bboxes[np.logical_and(pred_instance.labels == 0, pred_instance.scores > 0.3)]   #(pred_instance.labels == args.det_cat_id,pred_instance.scores > args.bbox_thr)
    bboxes = bboxes[nms(bboxes, 0.3), :4]

    # predict keypoints
    pose_results = inference_topdown(pose_estimator, img, bboxes)
    data_samples = merge_data_samples(pose_results)
    


    if visualizer is not None:
        visualizer.add_datasample(
            'result',
            img,
            data_sample=data_samples,
            draw_gt=False,
            draw_heatmap=False,
            draw_bbox=True,
            show_kpt_idx=True,
            skeleton_style="mmpose",
            show=False,
            wait_time=show_interval,
            kpt_thr=0.3)

    # if there is no instance detected, return None
    return data_samples.get('pred_instances', None)

In [5]:
#python demo/topdown_demo_with_mmdet.py --input S__14442525.jpg --output-root work_dirs --save-predictions --draw-bbox D:\mmdetection\configs\faster_rcnn\custom_model.py  D:\mmdetection\work_dirs\custom_model\epoch_12.pth configs\body_2d_keypoint\topdown_heatmap\coco\td-hm_res152_8xb32-210e_coco-384x288.py training_log/epoch_30.pth


detector = init_detector(
        r"D:\mmdetection\configs\faster_rcnn\custom_model.py", 
        r"D:\mmdetection\work_dirs\custom_model\epoch_12.pth",
        device="cuda:0")
detector.cfg = adapt_mmdet_pipeline(detector.cfg)

Loads checkpoint by local backend from path: D:\mmdetection\work_dirs\custom_model\epoch_12.pth


In [6]:
pose_estimator = init_pose_estimator(
        r"D:\mmpose\configs\body_2d_keypoint\topdown_heatmap\coco\td-hm_res152_8xb32-210e_coco-384x288.py",
        r"D:\mmpose\training_log\epoch_30.pth",
        device="cuda:0",
        )


Loads checkpoint by local backend from path: D:\mmpose\training_log\epoch_30.pth


In [20]:
print(pose_estimator)

TopdownPoseEstimator(
  (data_preprocessor): PoseDataPreprocessor()
  (backbone): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): ResLayer(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(

In [12]:
visualizer = VISUALIZERS.build(pose_estimator.cfg.visualizer)

visualizer.set_dataset_meta(
        pose_estimator.dataset_meta, skeleton_style="mmpose")

In [2]:

from mmpose.apis import MMPoseInferencer

inferencer = MMPoseInferencer(
    pose2d=r"D:\mmpose\configs\body_2d_keypoint\topdown_heatmap\coco\td-hm_res152_8xb32-210e_coco-384x288.py",
    pose2d_weights=r"D:\mmpose\training_log\epoch_30.pth",
#     det_model=r"D:\mmdetection\configs\faster_rcnn\custom_model.py",
#     det_weights=r"D:\mmdetection\work_dirs\custom_model\epoch_12.pth",
#     det_cat_ids=[0],  # the category id of 'human' class
#     device='cuda:0'
)



Loads checkpoint by local backend from path: D:\mmpose\training_log\epoch_30.pth
06/05 22:46:55 - mmengine - WARNING - Failed to search registry with scope "mmpose" in the "function" registry tree. As a workaround, the current "function" registry in "mmengine" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmpose" is a correct scope, or whether the registry is initialized.
06/05 22:46:55 - mmengine - WARNING - `Visualizer` backend is not initialized because save_dir is None.
Loads checkpoint by http backend from path: https://download.openmmlab.com/mmdetection/v3.0/rtmdet/rtmdet_m_8xb32-300e_coco/rtmdet_m_8xb32-300e_coco_20220719_112220-229f527c.pth
The model and loaded state dict do not match exactly

unexpected key in source state_dict: data_preprocessor.mean, data_preprocessor.std

06/05 22:46:56 - mmengine - WARNING - Failed to search registry with scope "mmdet" in the "function" registry tree. As a workaround, the

In [3]:
from mmpose.apis import MMPoseInferencer



cap = cv2.VideoCapture(r"D:\thesis_acu\experiment\keypoint_experiment\ear_reapaired_540x960\cather\_repaired.mp4")
count = 0
start = time.time()
while True:
    ret, frame = cap.read()
    result_generator = inferencer(frame, return_vis=True)
    print(result_generator)
    result = next(result_generator)
    print(result)
    out = result['visualization'][0]
#     print(out)
    im_rgb = cv2.cvtColor(out, cv2.COLOR_BGR2RGB)
    cv2.imshow('OpenCV Feed', im_rgb)
#     print(type(frame[0]))
#     print("-+-----------------------------")
#     print(out[0])
    print(87)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
    count += 1
end = time.time()       

        
cap.release()
cv2.destroyAllWindows()
print(format(end-start))   
print(count)

Output()

<generator object MMPoseInferencer.__call__ at 0x000001DB3E4C2F20>


Output()

defaultdict(<class 'list'>, {'visualization': [array([[[ 37,  39,  22],
        [ 33,  35,  18],
        [ 34,  36,  19],
        ...,
        [247, 249, 244],
        [247, 249, 244],
        [247, 249, 244]],

       [[ 37,  39,  22],
        [ 33,  35,  18],
        [ 34,  36,  19],
        ...,
        [247, 249, 244],
        [247, 249, 244],
        [247, 249, 244]],

       [[ 37,  39,  22],
        [ 34,  36,  19],
        [ 34,  36,  19],
        ...,
        [247, 249, 244],
        [247, 249, 244],
        [247, 249, 244]],

       ...,

       [[175, 118,  76],
        [175, 118,  76],
        [175, 118,  76],
        ...,
        [214, 164, 170],
        [212, 165, 170],
        [212, 165, 170]],

       [[174, 117,  75],
        [174, 117,  75],
        [174, 117,  75],
        ...,
        [216, 166, 174],
        [213, 168, 174],
        [213, 168, 174]],

       [[173, 116,  74],
        [173, 116,  74],
        [173, 116,  74],
        ...,
        [216, 166, 174],
  

defaultdict(<class 'list'>, {'visualization': [array([[[ 33,  39,  26],
        [ 29,  35,  22],
        [ 32,  35,  23],
        ...,
        [247, 249, 244],
        [247, 249, 244],
        [247, 249, 244]],

       [[ 33,  39,  26],
        [ 29,  35,  22],
        [ 32,  35,  23],
        ...,
        [247, 249, 244],
        [247, 249, 244],
        [247, 249, 244]],

       [[ 33,  39,  26],
        [ 30,  36,  23],
        [ 32,  35,  23],
        ...,
        [247, 249, 244],
        [247, 249, 244],
        [247, 249, 244]],

       ...,

       [[176, 119,  77],
        [176, 119,  77],
        [176, 119,  77],
        ...,
        [212, 162, 168],
        [212, 162, 168],
        [212, 162, 168]],

       [[175, 118,  76],
        [175, 118,  76],
        [175, 118,  76],
        ...,
        [216, 164, 170],
        [214, 164, 172],
        [214, 164, 172]],

       [[174, 117,  75],
        [174, 117,  75],
        [174, 117,  75],
        ...,
        [216, 164, 170],
  

...


87
<generator object MMPoseInferencer.__call__ at 0x000001DB3E4C2120>


defaultdict(<class 'list'>, {'visualization': [array([[[ 82,  98,  96],
        [ 81,  97,  95],
        [ 83,  99,  97],
        ...,
        [221, 244, 241],
        [221, 244, 241],
        [221, 244, 241]],

       [[ 79,  95,  93],
        [ 79,  95,  93],
        [ 83,  99,  97],
        ...,
        [221, 244, 241],
        [221, 244, 241],
        [221, 244, 241]],

       [[ 75,  91,  89],
        [ 77,  93,  91],
        [ 83,  99,  97],
        ...,
        [221, 244, 241],
        [221, 244, 241],
        [221, 244, 241]],

       ...,

       [[175, 140, 102],
        [177, 142, 104],
        [175, 140, 102],
        ...,
        [191, 172, 188],
        [192, 171, 188],
        [192, 171, 188]],

       [[177, 142, 104],
        [177, 142, 104],
        [175, 140, 102],
        ...,
        [193, 162, 182],
        [195, 164, 184],
        [197, 166, 186]],

       [[177, 142, 104],
        [177, 142, 104],
        [175, 140, 102],
        ...,
        [182, 151, 171],
  

Output()

87
<generator object MMPoseInferencer.__call__ at 0x000001DB40F02EB0>


defaultdict(<class 'list'>, {'visualization': [array([[[ 83,  99,  97],
        [ 84, 100,  98],
        [ 86, 102, 100],
        ...,
        [221, 244, 241],
        [221, 244, 241],
        [221, 244, 241]],

       [[ 82,  98,  96],
        [ 83,  99,  97],
        [ 85, 101,  99],
        ...,
        [221, 244, 241],
        [221, 244, 241],
        [221, 244, 241]],

       [[ 77,  93,  91],
        [ 81,  97,  95],
        [ 85, 101,  99],
        ...,
        [221, 244, 241],
        [221, 244, 241],
        [221, 244, 241]],

       ...,

       [[177, 142, 104],
        [178, 143, 105],
        [177, 142, 104],
        ...,
        [188, 172, 187],
        [191, 170, 187],
        [191, 170, 187]],

       [[178, 143, 105],
        [178, 143, 105],
        [177, 142, 104],
        ...,
        [187, 163, 181],
        [193, 165, 184],
        [195, 167, 186]],

       [[179, 144, 106],
        [179, 144, 106],
        [178, 143, 105],
        ...,
        [177, 153, 171],
  

In [4]:
import cv2
import numpy as np
import time
# Capturing the video file 0 for videocam else you can provide the url
capture = cv2.VideoCapture(0)
 
# Reading the first frame
_, frame1 = capture.read()
# Convert to gray scale
prvs = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
# Create mask
hsv_mask = np.zeros_like(frame1)
# Make image saturation to a maximum value
hsv_mask[..., 1] = 255
start = time.time()
count = 0
# Till you scan the video
while(1):
     
    # Capture another frame and convert to gray scale
    _, frame2 = capture.read()
#     next = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
 
#     # Optical flow is now calculated
#     flow = cv2.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
#     # Compute magnite and angle of 2D vector
#     mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
#     # Set image hue value according to the angle of optical flow
#     hsv_mask[..., 0] = ang * 180 / np.pi / 2
#     # Set value as per the normalized magnitude of optical flow
#     hsv_mask[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
#     # Convert to rgb
#     rgb_representation = cv2.cvtColor(hsv_mask, cv2.COLOR_HSV2BGR)
 
    cv2.imshow('frame2', frame2)
    kk = cv2.waitKey(20) & 0xff
#     # Press 'e' to exit the video
    if kk == ord('e'):
        break
#     # Press 's' to save the video
#     elif kk == ord('s'):
#         cv2.imwrite('Optical_image.png', frame2)
#         cv2.imwrite('HSV_converted_image.png', rgb_representation)
    prvs = next
    count += 1
end = time.time()
capture.release()
cv2.destroyAllWindows()
print(format(end-start))   
print(count)

12.751576900482178
381


In [7]:
import cv2
import numpy as np
def select_point(event, x, y, flags, params):
    global point, selected_point, old_points
    # record coordinates of mouse click
    if event == cv2.EVENT_LBUTTONDOWN:
        point = (x, y)
        selected_point = True
        old_points = np.array([[x, y]], dtype=np.float32)


# associate select function with window Selector



# initialize variables updated by function
selected_point = False
point = ()
old_points = ([[]])
cap = cv2.VideoCapture(r"D:\thesis_acu\experiment\keypoint_experiment\ear_reapaired_540x960\cather\_repaired.mp4")
ret, frame = cap.read()
# cv2.namedWindow('Optical Flow')
cv2.imshow('Optical Flow',frame)
cv2.setMouseCallback('Optical Flow',select_point)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [8]:
frame_gray_init = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

In [9]:
parameter_lucas_kanade = dict(winSize=(15, 15), maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

In [10]:
canvas = np.zeros_like(frame)

In [11]:
import time

cap = cv2.VideoCapture(r"D:\thesis_acu\experiment\keypoint_experiment\ear_reapaired_540x960\cather\_repaired.mp4")
count = 0
start = time.time()

while True:
    # get next frame
    ret, frame = cap.read()
    if frame is None:
        break
    # covert to grayscale
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    if selected_point is True:
        cv2.circle(frame, point, 5, (0, 0, 255), 2)
        # update object corners by comparing with found edges in initial frame
        new_points, status, errors = cv2.calcOpticalFlowPyrLK(frame_gray_init, frame_gray, old_points, None,
                                                         **parameter_lucas_kanade)

        # overwrite initial frame with current before restarting the loop
        frame_gray_init = frame_gray.copy()
        # update to new edges before restarting the loop
        old_points = new_points

        x, y = new_points.ravel()
        j, k = old_points.ravel()

        # draw line between old and new corner point with random colour
        canvas = cv2.line(canvas, (int(x), int(y)), (int(j), int(k)), (0, 255, 0), 3)
        # draw circle around new position
        frame = cv2.circle(frame, (int(x), int(y)), 5, (0, 255, 0), -1)

    result = cv2.add(frame, canvas)
    cv2.imshow('Optical Flow', result)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    count += 1
end = time.time()
print(format(end-start))
print(count)
cap.release()
cv2.destroyAllWindows()

8.279772281646729
573


In [ ]:
0:麻辣燙
1:丫頭
2:廣越
3:原味屋
4:鴨霸
5:港記

In [4]:
import numpy as np
np.random.randint(6, size=6)

array([0, 1, 0, 2, 4, 5])